# Setup

In [ ]:
import os
from dateutil.parser import parse

from osgeo import gdal
from osgeo import osr
from osgeo import ogr

import pandas as pd
import numpy as np
import utm

gdal.UseExceptions()
ogr.UseExceptions()

def utm_epsg_from_latlon(y, x):
    easting, northing, zone_num, zone_letter = utm.from_latlon(y, x)
    epsg = zone_num + 32600
    if (zone_letter == 'F'):
        epsg = epsg + 100
    return epsg

# Desired resolution for output images
out_res = 0.5
# Desired width of output images (in pixels)
width_pix = 10000
# Which spectral band to process
SPEC_TYPE = 'PAN'
# Maximum cloud cover fraction
MAX_CLOUD_COVER = .2

width_m = width_pix * out_res

#sites_shp = "O:/Data/GEF_Land_Degradation/Priority_sites/priority_sites_lund_ci.shp"
sites_shp = "O:/Data/Vital_Signs/Imagery/priority_sites_TEST.shp"
#footprints_shp = "O:/Data/GEF_Land_Degradation/LandDegradation_NCCS_Footprints20161207/LandDegradation_NCCS_Footprints20161207.shp"
footprints_shp = "O:/Data/Vital_Signs/Imagery/imagery_footprints.shp"

In [2]:
def get_site_geo(site):
    '''Get bounding box for site in UTM coordinates'''
    site_geom = site.geometry()
    lon, lat, z = site_geom.Centroid().GetPoint()
    utm_epsg = utm_epsg_from_latlon(lat, lon)

    to_sr = osr.SpatialReference()
    to_sr.ImportFromEPSG(utm_epsg)

    transform = osr.CoordinateTransformation(site_geom.GetSpatialReference(), to_sr)
    centroid = site_geom.Centroid()
    centroid.Transform(transform)
    easting, northing, z = centroid.GetPoint()

    northing = np.round(northing)
    easting = np.round(easting)
    ulx = easting - width_m/2
    uly = northing + width_m/2
    lrx  = easting + width_m/2
    lry = northing - width_m/2
    
    return((ulx, uly, lrx, lry, utm_epsg))

def get_site_bounding_box(site):
    ulx, uly, lrx, lry, utm_epsg = get_site_geo(site)

    to_sr = osr.SpatialReference()
    to_sr.ImportFromEPSG(utm_epsg)

    # Create the site bounding box
    ring = ogr.Geometry(ogr.wkbLinearRing)
    ring.AddPoint(ulx, uly)
    ring.AddPoint(lrx, uly)
    ring.AddPoint(lrx, lry)
    ring.AddPoint(ulx, lry)
    ring.AddPoint(ulx, uly)
    # Create polygon
    poly = ogr.Geometry(ogr.wkbPolygon)
    poly.AddGeometry(ring)
    
    poly.AssignSpatialReference(to_sr)
    
    return(poly)

def get_watermark(site):
    ulx, uly, lrx, lry, utm_epsg = get_site_geo(site)
    
    to_sr = osr.SpatialReference()
    to_sr.ImportFromEPSG(utm_epsg)

    # projWin --- subwindow in projected coordinates to extract: [ulx, uly, lrx, lry]
    #proj_window =  [ulx, uly, lrx, lry]

    ###############################################################################
    # Reproject watermark image to match this proj window

    # Fake the watermark image having the same extent as the desired utm output
    # image, but with a lower resolution. First calculate the necessary extent.
    wm_raw = gdal.Open('watermark.tif')
    wm_raw.SetProjection(to_sr.ExportToWkt())
    geo_raw = wm_raw.GetGeoTransform()
    wm_x_size = wm_raw.RasterXSize # Raster xsize
    wm_y_size = wm_raw.RasterYSize # Raster ysize
    wm_res = width_m / wm_x_size 
    geo_raw_utm = (ulx, wm_res, geo_raw[2], uly, geo_raw[4], -wm_res)

    # Setup new image in memory with extent matching the final output image
    wm_tmp_drv = gdal.GetDriverByName('MEM')
    wm_tmp = wm_tmp_drv.Create('', int((lrx - ulx)/wm_res), int((uly - lry)/wm_res), 1, gdal.GDT_Byte)
    wm_tmp.SetProjection(to_sr.ExportToWkt())
    wm_tmp.SetGeoTransform(geo_raw_utm)

    # Copy watermark image into new image with updated extent
    wm_tmp_band = wm_tmp.GetRasterBand(1)
    wm_band = wm_raw.GetRasterBand(1)
    wm_tmp_band.WriteArray(wm_band.ReadAsArray())

    # Reproject the watermark image to be at a resolution matching the final output
    # image.
    wm_drv = gdal.GetDriverByName('MEM')
    wm = wm_drv.Create('', int((lrx - ulx)/out_res), int((uly - lry)/out_res), 1, gdal.GDT_Byte)
    # Calculate the new geotransform
    geo_final = (ulx, out_res, geo_raw[2], uly, geo_raw[4], -out_res)
    # Set the geotransform
    wm.SetGeoTransform(geo_final)
    wm.SetProjection(to_sr.ExportToWkt())
    # Perform the projection/resampling 
    gdal.ReprojectImage(wm_tmp, wm)
    
    return(wm)

In [ ]:
# What fields are in the shapefile?
print "In sites:"
sites_layerdefn = sites.GetLayerDefn()
for i in range(sites_layerdefn.GetFieldCount()):
    print sites_layerdefn.GetFieldDefn(i).GetName()
    
print "In footprints:"
fp_layerdefn = fp.GetLayerDefn()
for i in range(fp_layerdefn.GetFieldCount()):
    print fp_layerdefn.GetFieldDefn(i).GetName()

# Main script

In [6]:
sites_drv = ogr.GetDriverByName("ESRI Shapefile")
sites_ds = sites_drv.Open(sites_shp, 0)
sites = sites_ds.GetLayer(0)
sites_sr = sites.GetSpatialRef()
site = sites.GetFeature(1)

fp_drv = ogr.GetDriverByName("ESRI Shapefile")
fp_ds = fp_drv.Open(footprints_shp, 0)
fp = fp_ds.GetLayer(0)
fp_sr = fp.GetSpatialRef()

sites.ResetReading() # For testing
for site in sites:
    aoi = get_site_bounding_box(site)

    # Get another aoi copy to reproject for spatial filtering
    aoi_reproj = get_site_bounding_box(site)
    aoi_reproj.Transform(osr.CoordinateTransformation(aoi.GetSpatialReference(), fp_sr))
    fp.SetSpatialFilter(aoi_reproj)
    
    # Setup watermark
    wm = get_watermark(site)
    wm_array = wm.ReadAsArray().astype(float)
    # Watermark is scaled by 255
    wm_array = wm_array / 255.
    
    fp.ResetReading() # For testing
    for footprint in fp:
        # Only process footprints that completely contain the aoi
        if not footprint.geometry().Contains(aoi_reproj):
            continue
        if footprint.GetField("SPEC_TYPE") != SPEC_TYPE:
            continue
        if footprint.GetField("CLOUDCOVER") > MAX_CLOUD_COVER:
            continue
        
        print footprint.GetField("S_FILEPATH")

        # Reproject raster to memory, cropping it in the process
        img = gdal.Open(footprint.GetField("S_FILEPATH"))
        ulx, uly, lrx, lry, utm_epsg = get_site_geo(site)
        gt = (ulx, out_res, 0, uly, 0, -out_res)
        
        img_tmp_drv = gdal.GetDriverByName('MEM')
        img_tmp = img_tmp_drv.Create('', int((lrx - ulx)/out_res), int((uly - lry)/out_res), 1, gdal.GDT_Float32)
        img_tmp.SetProjection(aoi.GetSpatialReference().ExportToWkt())
        img_tmp.SetGeoTransform(gt)
        
        img_reproj = gdal.Warp('', img, format='MEM', dstSRS=aoi.GetSpatialReference().ExportToWkt(),
                        outputBounds=(ulx, lry, lrx, uly), xRes=out_res, yRes=out_res)
        
        # Read as array to numpy to scale and watermark
        out_array = img_reproj.ReadAsArray().astype(float)
        # Scale array
        out_array = out_array - np.amin(out_array)
        out_array = (out_array / np.amax(out_array)) * 255
        
        # Watermark the image
        out_array = out_array * wm_array
        
        # Write watermarked image to disk. JPEG driver only supports
        # CreateCopy method, not Create, so need to write to an in-memory
        # ds first.
        img_out = gdal.GetDriverByName('MEM').Create('', img_reproj.RasterXSize, img_reproj.RasterYSize, 1, gdal.GDT_Byte)
        img_out.SetProjection(aoi.GetSpatialReference().ExportToWkt())
        img_out.SetGeoTransform(gt)
        img_out_band = img_out.GetRasterBand(1)
        img_out_band.WriteArray(out_array)
        
        img_date = parse(footprint.GetField("ACQ_TIME"))
        
        out_name = '%s_%s_%s.jpg'%(site.GetField("NAME"),
                                   img_date.strftime('%Y%m%d'),
                                   footprint.GetField("CATALOG_ID"))
        
        gdal.GetDriverByName("Jpeg").CreateCopy(out_name, img_out, options=["QUALITY=75"])

O:/Data/Vital_Signs/Imagery/ftp.digitalglobe.com/053974430130_01/053974430130_01_P001_PAN/14NOV19081000-P3DS-053974430130_01_P001.TIF
O:/Data/Vital_Signs/Imagery/ftp.digitalglobe.com/053974430090_01/053974430090_01_P001_PAN/14NOV16080040-P3DS-053974430090_01_P001.TIF
O:/Data/Vital_Signs/Imagery/ftp.digitalglobe.com/053974430100_01/053974430100_01_P001_PAN/14NOV16080105-P3DS-053974430100_01_P001.TIF


In [ ]:
import sys
sys.path.append('C:/OTB-5.10.1-win64/lib/python')
import otbApplication